<a href="https://colab.research.google.com/github/RTrombini/Projeto_Aluno_Especial_Unicamp/blob/main/Projeto_unicamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Projeto Information Retrieval**

 

O objetivo final desse projeto é fornecer um programa em que o usuário digite oque busca e receba de volta os documentos da coleção CISI que melhor se encaixam.

 

Os requisitos foram:

 

- Utilizar o algoritmo BM25

- Utilizar o diretorio CISI para testes




# **Metodologia**

 

Consultando o chatGPT sobre como seria a maneira mais eficiente de fazer um sistema de Information Retrival, obtivemos a seguinte sugestão de passos:

 

*1- Preparar os dados: Para criar um sistema de recuperação de informações, você precisará de um conjunto de dados contendo os documentos que deseja indexar. Esses documentos podem ser textos de sites da web, artigos, livros ou qualquer outra coisa que você queira indexar. Para este exemplo, vamos usar um conjunto de dados de artigos sobre tecnologia.*

 

*2- Indexar os documentos: Para indexar os documentos, você pode usar uma biblioteca de indexação, como o Whoosh ou o ElasticSearch. Essas bibliotecas são capazes de extrair informações dos documentos, como os termos que aparecem nos documentos e a frequência com que aparecem. Eles também permitem que você armazene essas informações em um formato que permita consultas rápidas.*

 

*3- Implementar o algoritmo BM25: Para implementar o algoritmo BM25, você pode usar a biblioteca python-whoosh, que fornece uma implementação do BM25. Você precisará calcular o BM25 para cada termo da consulta em relação a cada documento indexado. A biblioteca python-whoosh permite que você calcule o BM25 com base nos termos da consulta e nos termos indexados nos documentos.*

 

*5- Retornar os resultados: Depois de calcular o BM25 para cada documento e , você precisará retornar os resultados ao usuário. Os resultados podem ser classificados com base na pontuação BM25 ou em qualquer outra métrica que você preferir. Você pode retornar uma lista dos documentos mais relevantes ou exibir um snippet do texto em cada documento.*

 

Além de perguntar quais passos seriam recomendados para realizar a tarefa, perguntamos quais bibliotecas facilitariam o processo:

 

*1- Whoosh: É uma biblioteca de indexação em Python que permite indexar e pesquisar documentos em Python. É fácil de usar e oferece uma implementação do algoritmo BM25 para classificar documentos.*

 

*2- NLTK (Natural Language Toolkit): É uma biblioteca de processamento de linguagem natural em Python que fornece recursos para pré-processamento de texto, tokenização, análise sintática e outras tarefas relacionadas à linguagem natural.*

 

*3- Pandas: É uma biblioteca de análise de dados em Python que permite manipular e analisar dados em Python. É útil para carregar e manipular dados de documentos e conjuntos de dados.*

 

*4- Flask: É um framework web em Python que permite construir aplicativos web com facilidade. É útil para criar uma interface do usuário para o sistema de recuperação de informações.*

 

Como o projeto deve ser implementado no Google Colab, não há a necessidade de cosntruir um aplicativo com Flask e como já temos um dataset pre-definido (CISI), não iremos precisar utilizar o Pandas para manipulação dos dados.

 

Não consegui implementar um algoritmo com o NLTK para a ferramenta de buscas no colab pois não possuo experiencia previa com essa biblioteca nem com o colab e não teria tempo o bastante para fazer algo significativo com ela.

 

Apesar das demais sugestões do chatGPT, utilizaei apenas a priemeira lib sugerida (Whoosh) 

# **Instalação das biblitecas e arquivos necessários**


In [8]:
pip install whoosh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 17.1 MB/s eta 0:00:00


Primeiro precisamos fazer o download do arquivo CISI indicado para os testes. 

In [ ]:
import requests


url = 'http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz'
r = requests.get(url, allow_redirects=True)

In [ ]:
import tarfile


my_tar = tarfile.open('cisi.tar.gz')
my_tar.extractall('/content/drive/MyDrive/Colab_Notebooks') # specify which folder to extract to
my_tar.close()

Tendo o arquivo CISI ja descompactado no nosso diretorio, o proximo passo é a indexação de todo o conteudo do arquivo cisi.all.

# **Indexação**

 

Tendo o arquivo CISI ja descompactado no nosso diretorio, o proximo passo é a indexação de todo o conteudo do arquivo cisi.all.

 

Para isso, o importamos as bibliotecas necessárias (os, re, whoosh), definimos o esquema dos campos do índice e cria um indexador para a pasta do índice.

 

Depois, o lemos o arquivo de texto da coleção CISI e usamos expressões regulares para separar os documentos e extrair os campos relevantes (ID, título, autor e resumo). Em seguida, adiciona cada documento ao índice usando o método "add_document()" do objeto "writer".

 

Por fim, o código finaliza a indexação com o método "commit()" do objeto "writer".

 

O resultado final é um índice que pode ser usado para pesquisar informações em documentos da coleção CISI.

 

In [ ]:
import os
import re
from whoosh import index
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID


def index_cisi_collection():

    if index.exists_in("/content/drive/MyDrive/Colab_Notebooks/cisi_index"):
        try:
            ix = index.open_dir("/content/drive/MyDrive/Colab_Notebooks/cisi_index")
            writer = ix.writer()
        except index.LockError:
            ix = index.open_dir("/content/drive/MyDrive/Colab_Notebooks/cisi_index", readonly=True)
            ix._unlock()
            os.remove(os.path.join("/content/drive/MyDrive/Colab_Notebooks/cisi_index", "write.lock"))
            ix = index.open_dir("/content/drive/MyDrive/Colab_Notebooks/cisi_index")
            writer = ix.writer()
    else:
        # Define o schema para os campos do índice
        schema = Schema(id=ID(unique=True, stored=True),
                        title=TEXT(stored=True),
                        author=TEXT(stored=True),
                        abstract=TEXT(stored=True))

        # Cria o diretório do índice (se ele não existir)
        if not os.path.exists("/content/drive/MyDrive/Colab_Notebooks/cisi_index"):
            os.mkdir("/content/drive/MyDrive/Colab_Notebooks/cisi_index")

        # Cria um indexador para a pasta do índice
        ix = create_in("/content/drive/MyDrive/Colab_Notebooks/cisi_index", schema)
        writer = ix.writer()

    # Abre o arquivo de texto da coleção CISI
    with open("/content/drive/MyDrive/Colab_Notebooks/cisi/CISI.ALL", "r") as f:
        # Lê o arquivo inteiro de uma vez
        text = f.read()

        # Usa expressões regulares para separar os documentos e extrair os campos relevantes
        pattern = r"\.I (\d+)\n\.T\n(.*?)\n\.A\n(.*?)\n\.W\n(.*?)\n(?:\.X\n(.*?))?(?=\n\.I|\Z)"
        docs = re.findall(pattern, text, re.DOTALL)

        for doc in docs:
            doc_id, title, author, abstract, content, = doc

            #Adiciona o documento ao índice
            writer.add_document(id=doc_id,
                                title=title,
                                author=author,
                                abstract=abstract)

    # Finaliza a indexação
    writer.commit()


# **Input do Usuário**

 

Este código define uma função chamada highlights() que pede ao usuário para digitar uma frase e retorna as palavras mais importantes dessa frase, ignorando palavras comuns em inglês (como artigos e preposições) e pontuações. O ingles foi escolhido pois os arquivos da pasta cisi estão em inglês.

 

A biblioteca string é importada para utilizar a variável punctuation, que é uma string contendo todos os caracteres de pontuação.

 

O código começa pedindo ao usuário para inserir uma frase e, em seguida, inicializa uma lista vazia chamada results que será preenchida com as cinco palavras mais importantes. Em seguida, é criada uma lista de stop words, que contém palavras comuns em inglês que serão ignoradas. O dicionário word_counts é inicializado para contar a frequência das palavras e as palavras da entrada do usuário são convertidas para minúsculas e divididas em uma lista de palavras.

 

O código percorre cada palavra da lista, ignorando as stop words e pontuações e, em seguida, conta sua frequência no dicionário word_counts. Depois que todas as palavras são contadas, a lista é ordenada em ordem decrescente de frequência e as cinco palavras mais importantes são adicionadas à lista results.

 

Finalmente, a função retorna uma lista com as palavras-chave da entrada fornecida pelo usuário.

 

In [9]:
import string

def highligths():
    # Obter a entrada do usuário
    user_input = input("Digite uma frase: ")
    results = []

    # Definir uma lista de palavras comuns em ingles a serem ignoradas
    stop_words = ['a', 'an', 'the', 'in', 'on', 'at', 'to', 'for', 'of', 'with']

    # Inicializar um dicionário vazio para contar a frequência das palavras
    word_counts = {}

    # Dividir a entrada do usuário em palavras e converter para letras minúsculas
    words = user_input.lower().split()

    # Percorrer todas as palavras e contar sua frequência, ignorando as stop words e pontuações
    for word in words:
        if word not in stop_words and word not in string.punctuation:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1

    # Ordenar as palavras por frequência
    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Criar uma lista com as palavras mais importantes
    for word, count in sorted_words:
        results.append(word)

    # Retorna a lista com as palavras mais importantes
    return results


# **Busca**

 

Esse codigo utiliza, novamente, a biblioteca whoosh para fazer a busca nos documentos ja indexados. Especificamente, ele importa a função open_dir do módulo index, e várias classes e funções do módulo scoring, qparser, e query.

 

O código define uma função parser que realiza uma busca em um índice de documentos de texto previamente criado, que está localizado na pasta cisi_index. A função recebe uma lista de strings de busca (search_strings) e retorna os resultados da busca.

 

A função usa um objeto MultifieldParser para definir um esquema de busca com três campos - "abstract", "author" e "id" - e um objeto BM25F para definir o algoritmo de pontuação. Em seguida, ela itera sobre as strings de busca fornecidas, criando uma query para cada uma usando o método parse do objeto parser, e as une usando o operador Or. Por fim, a função executa a busca usando o método search do objeto searcher, iterando sobre os resultados e imprimindo as informações relevantes de cada documento.

 

In [10]:
from whoosh.index import open_dir
from whoosh import scoring, qparser
from whoosh.query import Or


def parser(search_strings):
    # Abre o índice de busca salvo na pasta 'cisi_index'
    ix = open_dir("/content/drive/MyDrive/Colab_Notebooks/cisi_index")

    # Define o esquema de consulta com três campos: abstract, author e id
    schema = ix.schema
    parser = qparser.MultifieldParser(["abstract", "author", "id"], schema=schema)

    # Define o algoritmo BM25 como o algoritmo de pontuação
    searcher = ix.searcher(weighting=scoring.BM25F())

    # Cria uma lista de queries a partir das strings de busca fornecidas
    queries = []
    for search_string in search_strings:
        queries.append(parser.parse(search_string))

    # Une as queries em uma única query usando o operador OR
    query = Or(queries)

    # Executa a consulta
    results = searcher.search(query, limit=None)

    # Itera sobre os resultados e imprime as informações relevantes de cada documento
    for hit in results:
        print(f"ID: {hit['id']}")
        print(f"Title: {hit['title']}")
        print(f"Author: {hit['author']}")
        print(f"Abstract: {hit['abstract']}")
        print(f"Score: {hit.score}")
        print("")
        print("__________________________//____________________")
        print("")


# **Rodando o Script**

Ao rodar a célula abaixo, o usuário irá ser indagado a digitar uma entrada para o sistema. 

O código checa se já existe um diretorio com o arquivo CISI indexado e, caso contrário, faz a indexação com a função *index_cisi_collection()*.

Após isso, a função highligths() recebe a entrada do usuário e estima quais as palavras mais relevantes dessa entrada. O resultado dessa função é uma lista que será a entrada da função que realizará a busca em si, utilizando o algoritimo BM25, *parser()*.

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Colab_Notebooks/cisi_index"):
    index_cisi_collection()

query = highligths()

parser(query)